### 資料集簡介
<p>欄位說明:</p>
<p>FileID: 檔案識別ID</p>
<p>CustomerID: 使用者裝置識別ID</p>
<p>QueryTs: 該筆資料發生時間</p>
<p>ProductID: 使用者裝置的產品代碼</p>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
plt.rcParams['font.family']='SimHei' #顯示中文

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load in the train datasets
data_dir = "/data/examples/trend/data/"
train = pd.read_csv(data_dir + 'training-set.csv', encoding = "utf-8", header=None)
test = pd.read_csv(data_dir + 'testing-set.csv', encoding = "utf-8", header=None)

In [4]:
#query_log裡面被官方排除的 FileID
train_exc = pd.read_csv(data_dir + 'exception/exception_train.txt', encoding = "utf-8", header=None)
test_exc = pd.read_csv(data_dir + 'exception/exception_testing.txt', encoding = "utf-8", header=None)

In [5]:
test_exc.head(2)

,0
0,4eedf630f7160dafea969e5d57239d31
1,8e1c12f42b705cb465bab88225d03c81


In [6]:
# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values='NaN', axis=0, strategy='mean') 
# imputer.fit_transform(X[:,[1,3]])

In [7]:
# training set - label: 0:非惡意程式, 1:惡意程式
train.columns=['FileID','label']
train.head(2)

,FileID,label
0,0000e2398b12121a85166fed5fe2a3da,0
1,0001fe8dce14ce099aa6ca8ea5026ea7,0


In [8]:
# testing set - AUC: Area Under ROC Curve
test.columns=['FileID','AUC']
test.head(2)

,FileID,AUC
0,00008c73ee43c15b16c26b26398c1577,0.5
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.5


In [9]:
#確認排除的FileID在training set裡面找不到
for item in train_exc:
    print(train[train['FileID']==item])

Empty DataFrame
Columns: [FileID, label]
Index: []


In [10]:
#取0301當天的query log來查看
log_files = os.listdir(data_dir+"query_log")
query_logs = []
for log_file in log_files:
    query_log = pd.read_csv(data_dir + 'query_log/' + log_file, encoding = "utf-8", header=None)
    query_log.columns=['FileID','CustomerID','QueryTs','ProductID']
    query_log['times'] = 1
    query_logs.append(query_log)

In [11]:
querys = pd.concat(query_logs)

In [12]:
querys.describe()

,QueryTs,times
count,8.327311e+07,83273110.0
mean,1.492394e+09,1.0
std,2.138802e+06,0.0
min,1.488326e+09,1.0
25%,1.490234e+09,1.0
50%,1.492291e+09,1.0
75%,1.494531e+09,1.0
max,1.496275e+09,1.0


In [13]:
querys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83273110 entries, 0 to 563303
Data columns (total 5 columns):
FileID        object
CustomerID    object
QueryTs       int64
ProductID     object
times         int64
dtypes: int64(2), object(3)
memory usage: 20.2 GB


### 樞杻分析

In [ ]:
query_0301.pivot_table(values='times',index=['FileID'],columns='ProductID',aggfunc='sum')

### 聚合函數
count（個數）, sum（加總）, mean（平均）, median（中位數）, std（標準差）, var（變異數）, first（第一個非NA）, last（最後一個非NA）

In [ ]:
query_0301.groupby(['FileID','CustomerID','ProductID'])[['times']].sum()

### 其它會用到的工具

In [ ]:
#AUC計算範例
import numpy as np
from sklearn import metrics
y = np.array([1, 1, 2, 2])
pred = np.array([0.9, 0.1, 0.7, 0.9])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
metrics.auc(fpr, tpr)


In [ ]:
#timestamp轉換
import datetime
print(
    datetime.datetime.fromtimestamp(
        int("1488326402")
    ).strftime('%Y-%m-%d %H:%M:%S'))
